In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(edgeR)
    library(ComplexHeatmap)
    library(viridis)
    library(ggplot2)
    library(tidyverse)
    library(doParallel)
    library(foreach)
    library(clusterProfiler)
    library(org.Mm.eg.db)
    library(dplyr)
})

options(future.globals.maxSize = Inf)
options(ggrepel.max.overlaps = Inf)

### DEG of mature mice bipolar cells vs. young mice bipolar cells

In [2]:
count_dir <- "results/age_deg_edgeR/counts/"
meta_dir <- "results/age_deg_edgeR/meta_data/"
cell_types <- c('rod bipolar cell','type 6 cone bipolar cell (sensu Mus)', 'type 5a cone bipolar cell', 'type 3b cone bipolar cell',
 'type 5 cone bipolar cell (sensu Mus)', 'type 5b cone bipolar cell',
 'type 9 cone bipolar cell (sensu Mus)',
 'type 3a cone bipolar cell',
 'type 8 cone bipolar cell (sensu Mus)',
 'type 7 cone bipolar cell (sensu Mus)',
 'type 4 cone bipolar cell (sensu Mus)',
 'type 1 cone bipolar cell (sensu Mus)',
 'type 2 cone bipolar cell (sensu Mus)')

In [4]:
for (ct in cell_types){
    counts_name <- paste0(count_dir, sprintf("counts_%s.csv", ct))
    meta_name <- paste0(meta_dir, sprintf("meta_%s.csv", ct))
    fn <- paste0("results/W8_vs_P17_DEG/", sprintf("%s_DEG_W8_vs_P17.csv", ct))
    counts <- read.csv(counts_name, row.names = 1)
    meta   <- read.csv(meta_name)
    
    # Ensure sample names match
    stopifnot(all(meta$sample == colnames(counts)))
    
    # Build DGEList
    group <- factor(meta$age, levels = c("P17", "W8"))
    dge <- DGEList(counts = as.matrix(counts), group = group)
    
    # This step handles imbalance properly
    keep <- filterByExpr(dge, group = group)
    dge <- dge[keep,,keep.lib.sizes = FALSE]
    
    # Normalize
    dge <- calcNormFactors(dge)
    
    # Design matrix and test
    design <- model.matrix(~ group)
    dge <- estimateDisp(dge, design)
    fit <- glmQLFit(dge, design)
    qlf <- glmQLFTest(fit, coef = 2)
    result <- topTags(qlf, n = Inf)$table

    write.csv(result,fn, row.names = TRUE)
}

### Plot DEG as volcano plot

In [ ]:
deg_dir <- "results/W8_vs_P17_DEG/"

In [14]:
for (file in list.files(deg_dir, pattern = "\\.csv$")){
    # Read the CSV
    df <- read.csv(file.path(deg_dir,file))
    
    # Add 1e-300 to FDR == 0 to avoid log(0)
    df$FDR[df$FDR == 0] <- 1e-400
    
    # Calculate -log10(FDR)
    df$log10FDR <- -log10(df$FDR)
    
    # Label colors based on FDR threshold
    df$color <- ifelse(df$FDR < 0.05, "red", "black")
    
    # Generate plot
    p <- ggplot(df, aes(x = logFC, y = log10FDR, color = color)) +
    geom_point() +
    scale_color_identity() +
    labs(
      title = basename(file),
      x = "logFC",
      y = "-log10(FDR)"
    ) +
    theme_minimal() +
    theme(
      panel.background = element_rect(fill = "white", color = NA),
      plot.background = element_rect(fill = "white", color = NA)
    )
    
    # Save the plot as a PNG
    outname <- paste0("results/DEG_volcano_plots/",file, "_volcano.png")
    ggsave(outname, plot = p, height = 7, width = 14)
}

### GO analysis

In [2]:
deg_dir <- "results/W8_vs_P17_DEG/"
up_dir <- "results/W8_vs_P17_GO_up/"
down_dir <- "results/W8_vs_P17_GO_down/"

In [3]:
up_cluster_degs <- list()
down_cluster_degs <- list()
for (file in list.files(deg_dir)){
    df <- read.csv(paste0(deg_dir, file), row.names = 1)
    ct <- strsplit(file, split = "_")[[1]][1]
    
    # up_degs <- rownames(df[df$FDR < 0.05 & df$logFC > 0,])
    # up_degs <- sub("\\..*", "", up_degs) # Remove whatever after . in gene name indicating gene version
    # mapped <- bitr(up_degs, fromType="ENSEMBL", toType="SYMBOL", OrgDb=org.Mm.eg.db) # Filter for mappable genes
    # mapped_unique <- mapped %>% distinct(ENSEMBL, .keep_all = TRUE)
    # up_degs_cleaned <- mapped_unique$ENSEMBL
    # up_cluster_degs[[ct]] <- up_degs_cleaned
    up_cluster_degs[[ct]] <- rownames(df[df$FDR < 0.05 & df$logFC > 0,])

    # down_degs <- rownames(df[df$FDR < 0.05 & df$logFC < 0,])
    # down_degs_cleaned <- sub("\\..*", "", down_degs)
    # mapped <- bitr(down_degs, fromType="ENSEMBL", toType="SYMBOL", OrgDb=org.Mm.eg.db)
    # mapped_unique <- mapped %>% distinct(ENSEMBL, .keep_all = TRUE)
    # down_degs_cleaned <- mapped_unique$ENSEMBL
    # down_cluster_degs[[ct]] <- down_degs_cleaned
    down_cluster_degs[[ct]] <- rownames(df[df$FDR < 0.05 & df$logFC < 0,])
}

In [4]:
bkgnd_df <- read.csv("results/expressed_genes.csv")
bkgnd <- bkgnd_df$expressed_genes
# bkgnd <- sub("\\..*", "", bkgnd)
# mapped <- bitr(bkgnd, fromType="ENSEMBL", toType="SYMBOL", OrgDb=org.Mm.eg.db)
# mapped_unique <- mapped %>% distinct(ENSEMBL, .keep_all = TRUE)
# bkgnd <- mapped_unique$ENSEMBL

In [5]:
length(bkgnd)

[1] 27525

In [6]:
pval_thresh <- 0.05

In [7]:
cell_types = names(up_cluster_degs)

In [8]:
head(cell_types)

[1] "rod bipolar cell"                    
[2] "type 1 cone bipolar cell (sensu Mus)"
[3] "type 2 cone bipolar cell (sensu Mus)"
[4] "type 3a cone bipolar cell"           
[5] "type 3b cone bipolar cell"           
[6] "type 4 cone bipolar cell (sensu Mus)"

In [9]:
for (ct in cell_types){
    up_genes <- up_cluster_degs[[ct]]
    if (length(up_genes) == 0){
        next
    }
    
    curr.result <- enrichGO(
        gene = up_genes,
        universe = bkgnd,
        keyType = "ENSEMBL",
        OrgDb = org.Mm.eg.db,
        ont = "BP",
        pvalueCutoff = pval_thresh,
        readable = TRUE
    )
    curr.result.simplified <- clusterProfiler::simplify(curr.result)
    file_name <- file.path(up_dir, paste0("GO_", ct, "_up.csv"))
    write.csv(
        curr.result.simplified@result,
        file_name,
        row.names = FALSE
    )
}

In [9]:
for (ct in cell_types){
    down_genes <- down_cluster_degs[[ct]]
    if (length(down_genes) == 0){
        next
    }

    curr.result <- enrichGO(
        gene = down_genes,
        universe = bkgnd,
        keyType = "ENSEMBL",
        OrgDb = org.Mm.eg.db,
        ont = "BP",
        pvalueCutoff = pval_thresh,
        readable = TRUE
    )
    curr.result.simplified <- clusterProfiler::simplify(curr.result)
    file_name <- file.path(down_dir, paste0("GO_", ct, "_down.csv"))
    write.csv(
        curr.result.simplified@result,
        file_name,
        row.names = FALSE
    )
}

### Plot dotplot for GO function

In [22]:
# Set your folder with .csv files
csv_path <- "results/W8_vs_P17_GO_up/"

# List all csv files
csv_files <- list.files(csv_path, pattern = "\\.csv$", full.names = TRUE)

# Read and combine all files with cell type as ID
go_data <- lapply(csv_files, function(file) {
    df <- read_csv(file)
    s1 <- strsplit(file, split = "/")[[1]][4]
    df$cell_type <- strsplit(s1, split = "_")[[1]][2]
    df <- df %>% slice_head(n = 30)
    return(df)
}) %>% bind_rows()

# Add -log10(p.adjust) column
go_data <- go_data %>% mutate(log10p.adjust = -log10(p.adjust))

go_data_filtered <- go_data %>%
  group_by(Description) %>%
  filter(n_distinct(cell_type) > 2) %>%
  ungroup()

# # Optional: Cap large -log10(p.adjust) for better visual scaling
# go_data_filtered$log10p.adjust <- pmin(go_data_filtered$log10p.adjust, 50)

# Plot
p <- ggplot(go_data_filtered, aes(x = cell_type, y = Description, size = Count, color = log10p.adjust)) +
    geom_point() +
    scale_color_viridis(name = "-log10(p.adjust)", option = "C", direction = -1) +
    scale_size_continuous(name = "Gene Count", range = c(1,8)) +
    theme_minimal(base_size = 11) +
    theme(
    axis.text.x = element_text(angle = 90, hjust = 1),
    axis.text.y = element_text(size = 12),
    axis.title.x = element_blank(),
    axis.title.y = element_blank(),
    aspect.raio = 0.4
    ) +
    labs(title = "GO Term Enrichment by Cell Type")

plot_name <- "results/W8_vs_P17_GO_up_plot.png"
ggsave(plot_name, plot = p, height = 15, width = 12, bg = "white")

Rows: 195 Columns: 12
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): ID, Description, GeneRatio, BgRatio, geneID
dbl (7): RichFactor, FoldEnrichment, zScore, pvalue, p.adjust, qvalue, Count

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 1 Columns: 12
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): ID, Description, GeneRatio, BgRatio, geneID
dbl (7): RichFactor, FoldEnrichment, zScore, pvalue, p.adjust, qvalue, Count

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types

In [23]:
# Set your folder with .csv files
csv_path <- "results/W8_vs_P17_GO_down/"

# List all csv files
csv_files <- list.files(csv_path, pattern = "\\.csv$", full.names = TRUE)

# Read and combine all files with cell type as ID
go_data <- lapply(csv_files, function(file) {
    df <- read_csv(file)
    s1 <- strsplit(file, split = "/")[[1]][4]
    df$cell_type <- strsplit(s1, split = "_")[[1]][2]
    df <- df %>% slice_head(n = 30)
    return(df)
}) %>% bind_rows()

# Add -log10(p.adjust) column
go_data <- go_data %>% mutate(log10p.adjust = -log10(p.adjust))

go_data_filtered <- go_data %>%
  group_by(Description) %>%
  filter(n_distinct(cell_type) > 2) %>%
  ungroup()

# # Optional: Cap large -log10(p.adjust) for better visual scaling
# go_data_filtered$log10p.adjust <- pmin(go_data_filtered$log10p.adjust, 50)

# Plot
p <- ggplot(go_data_filtered, aes(x = cell_type, y = Description, size = Count, color = log10p.adjust)) +
    geom_point() +
    scale_color_viridis(name = "-log10(p.adjust)", option = "C", direction = -1) +
    scale_size_continuous(name = "Gene Count", range = c(1,8)) +
    theme_minimal(base_size = 11) +
    theme(
    axis.text.x = element_text(angle = 90, hjust = 1),
    axis.text.y = element_text(size = 12),
    axis.title.x = element_blank(),
    axis.title.y = element_blank(),
    aspect.raio = 0.4
    ) +
    labs(title = "GO Term Enrichment by Cell Type")

plot_name <- "results/W8_vs_P17_GO_down_plot.png"
ggsave(plot_name, plot = p, height = 15, width = 12, bg = "white")

Rows: 115 Columns: 12
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): ID, Description, GeneRatio, BgRatio, geneID
dbl (7): RichFactor, FoldEnrichment, zScore, pvalue, p.adjust, qvalue, Count

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 107 Columns: 12
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (5): ID, Description, GeneRatio, BgRatio, geneID
dbl (7): RichFactor, FoldEnrichment, zScore, pvalue, p.adjust, qvalue, Count

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column typ